In [ ]:
!pip install tensorflow tensorflow-datasets scikit-learn numpy seaborn matplotlib pillow

#
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image

#
IMG_SIZE = (160,160)
BATCH_SIZE = 32

(train_ds, val_ds), info = tfds.load(
    'tf_flowers',
    split=['train[:85%]', 'train[85%:]'],
    with_info=True,
    as_supervised=True
)

def preprocess(img, label):
    img = tf.image.resize(img, IMG_SIZE) / 255.0
    return img, label

train_ds = train_ds.map(preprocess).batch(BATCH_SIZE).prefetch(1)
val_ds = val_ds.map(preprocess).batch(BATCH_SIZE).prefetch(1)
class_names = info.features['label'].names

#Get one batch
images, labels = next(iter(train_ds))
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)
print("Class Names:", class_names)



#
base = tf.keras.applications.MobileNetV2(
    input_shape=(*IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base.trainable = False

model = tf.keras.Sequential([
    base, 
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history1 = model.fit(train_ds, validation_data=val_ds, epochs=3, verbose=1)

base.summary()
model.summary()

base.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history2 = model.fit(train_ds, validation_data=val_ds, epochs=3, verbose=1)



#
train_acc = history1.history['accuracy'] + history2.history['accuracy']
val_acc = history1.history['val_accuracy'] + history2.history['val_accuracy']
train_loss = history1.history['loss'] + history2.history['loss']
val_loss = history1.history['val_loss'] + history2.history['val_loss']

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(train_acc, label='train')
plt.plot(val_acc, label='val')
plt.title('Accuracy');
plt.legend()

plt.subplot(1,2,2)
plt.plot(train_loss, label='train')
plt.plot(val_loss, label='val')
plt.title('Loss');
plt.legend()
plt.show();

loss, acc = model.evaluate(val_ds, verbose=0)
print('Validation Accuracy: ',acc)

y_true, y_pred = [], []
for x, y in val_ds:
    y_true.extend(y.numpy())
    y_pred.extend(np.argmax(model.predict(x, verbose=0), axis = 1))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix'); plt.ylabel('True Label'); plt.xlabel('Predicted Label')
plt.show()

print(classification_report(y_true, y_pred, target_names=class_names))


#
base = tf.keras.applications.MobileNetV2(
    input_shape=(*IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base.trainable = False

model = tf.keras.Sequential([
    base, 
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history1 = model.fit(train_ds, validation_data=val_ds, epochs=3, verbose=1)

base.summary()
model.summary()

base.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history2 = model.fit(train_ds, validation_data=val_ds, epochs=3, verbose=1)


#
train_acc = history1.history['accuracy'] + history2.history['accuracy']
val_acc = history1.history['val_accuracy'] + history2.history['val_accuracy']
train_loss = history1.history['loss'] + history2.history['loss']
val_loss = history1.history['val_loss'] + history2.history['val_loss']

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(train_acc, label='train')
plt.plot(val_acc, label='val')
plt.title('Accuracy');
plt.legend()

plt.subplot(1,2,2)
plt.plot(train_loss, label='train')
plt.plot(val_loss, label='val')
plt.title('Loss');
plt.legend()
plt.show();

loss, acc = model.evaluate(val_ds, verbose=0)
print('Validation Accuracy: ',acc)

y_true, y_pred = [], []
for x, y in val_ds:
    y_true.extend(y.numpy())
    y_pred.extend(np.argmax(model.predict(x, verbose=0), axis = 1))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix'); plt.ylabel('True Label'); plt.xlabel('Predicted Label')
plt.show()

print(classification_report(y_true, y_pred, target_names=class_names))


#
test_folders = Path('Test Dataset')
img_types = ("*.jpg", "*.jpeg", "*.png", "*.webp")

def predict(img_path):
    img = Image.open(img_path).convert('RGB').resize(IMG_SIZE)
    arr = np.array(img)[np.newaxis] / 255.0
    pred = model.predict(arr, verbose=0)[0]
    cls = class_names[np.argmax(pred)]
    conf = np.max(pred)
    print(f"{img_path.name} -> {cls} ({conf:.3f})")

if test_folders.exists():
    for pattern in img_types:
        for img_path in test_folders.glob(pattern):
            predict(img_path)
else:
    print("Test folder not found.")